### Agentic AI - Multi Agent Design for Content Creation using Llama and Autogen

Here, a content creation system is created using autogen's AssistantAgent and ConversableAgent methods. The process is referred from [pdichone](https://github.com/pdichone). The content creation is done using multiple agents woring as a team involving writer, SEO reviewer, Compliance reviewer etc.

#### Importing needed libraries

In [1]:
import os
from autogen import ConversableAgent, AssistantAgent
from typing import Annotated

#### Creating LLM Configuration 

Llama3.2 local LLM is used through Ollama.

In [40]:
config_list = [
    {
        "model": "llama3.2:latest",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama"
    },
]
llm_config = {"config_list": config_list, "temperature": 0.0}

#### Creating task prompt

In [42]:
task = """
        Write a concise, engaging article about
       AI Agentic Workflows. Make sure the article is
       within 350 words.
       """

#### Creating Assistant Agent

AssistantAgent is initialized with a system prompt with details about what is expected from the agent as a writer.

In [43]:
writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
    "articles (with title) on given topics. You must polish your "
    "writing based on the feedback you receive and give a refined "
    "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

#### Generation response from Writer Agent
By providing a user prompt to the agent's generate_reply method, the response is retrieved.

In [44]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

[autogen.oai.client: 12-10 22:31:45] {732} WARNING - Model llama3.2:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [45]:
reply

"**Unlocking Efficiency: The Power of AI Agentic Workflows**\n\nIn today's fast-paced business landscape, organizations are constantly seeking ways to streamline processes and boost productivity. One innovative solution gaining traction is AI agentic workflows. By harnessing the power of artificial intelligence (AI), companies can create dynamic, adaptive workspaces that optimize efficiency and drive success.\n\n**What are AI Agentic Workflows?**\n\nAI agentic workflows refer to a type of workflow management system that leverages machine learning algorithms to analyze and adapt to changing business needs. These systems use natural language processing (NLP) and predictive analytics to identify patterns, predict outcomes, and make data-driven decisions in real-time.\n\n**Benefits of AI Agentic Workflows**\n\nThe benefits of AI agentic workflows are numerous:\n\n* **Increased Efficiency**: By automating routine tasks and streamlining processes, organizations can free up resources for more

#### Critic Agent Creation
The critic agent will iteratively review and improve the article generated by writer agent. It takes in a termination argument to know when to terminate the review process.

In [46]:
critic = AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
    "the writer and provide constructive "
    "feedback to help improve the quality of the content.",
)

#### Initiate the agent to agent chat

The critic object has 'initiate_chat' method that can be used to give the writing task to the writer and then follow up with reviews.

In [47]:
res = critic.initiate_chat(
    recipient=writer, message=task, max_turns=3, summary_method="last_msg"
)

Critic (to Writer):


        Write a concise, engaging article about
       AI Agentic Workflows. Make sure the article is
       within 350 words.
       

--------------------------------------------------------------------------------
[autogen.oai.client: 12-10 22:33:11] {732} WARNING - Model llama3.2:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Writer (to Critic):

**Unlocking Efficiency: The Power of AI Agentic Workflows**

In today's fast-paced business landscape, organizations are constantly seeking ways to streamline processes and boost productivity. One innovative solution gaining traction is AI agentic workflows. By harnessing the power of artificial intelligence (AI), companies can create dynamic, adaptive workspaces that optimize efficiency and drive success.

**What are AI Agentic Workflows?**

AI agentic workflows refer to a type of workflow management syst

#### Creating multiple Reviewer agents

Each agent is an instance of AssistantAgent with unique system message explaining the tasks expected out of each agent.

In [48]:
SEO_reviewer = AssistantAgent(
    name="SEO-Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
    "your ability to optimize content for search engines, "
    "ensuring that it ranks well and attracts organic traffic. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role, like 'SEO Reviewer:'.",
)

compliance_reviewer = AssistantAgent(
    name="Compliance-Reviewer",
    llm_config=llm_config,
    system_message="You are a compliance reviewer. You ensure that the content "
    "adheres to the guidelines and regulations of the industry and Google algorithms. "
    "Begin the review by stating your role, like 'Compliance Reviewer:'.",
)

meta_reviewer = AssistantAgent(
    name="Meta-Reviewer",
    llm_config=llm_config,
    system_message="You are a meta-reviewer. You provide a final review of the content, "
    "ensuring that all the feedback from the previous reviewers has been incorporated. "
    "Begin the review by stating your role, like 'Meta Reviewer:'.",
)


def reflection_message(recipient, messages, sender, config):
    return f"""Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"""

#### Creation of nested chat formats

In [52]:
review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
        },
        "max_turns": 1,
    },
    {
        "recipient": compliance_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}.",
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1,
    },
]


#### Register Nested Chats
When the writed provides a response, it will trigger the nested chats to initiate the review process in the predefined format.

In [53]:
critic.register_nested_chats(review_chats, trigger=writer)

#### Initiate the Chat 
This initiates the chat and further review processes.

In [54]:
res = critic.initiate_chat(
    recipient=writer, message=task, max_turns=2, summary_method="last_msg"
)

Critic (to Writer):


        Write a concise, engaging article about
       AI Agentic Workflows. Make sure the article is
       within 350 words.
       

--------------------------------------------------------------------------------
[autogen.oai.client: 12-10 23:29:02] {732} WARNING - Model llama3.2:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Writer (to Critic):

**Unlocking Efficiency: The Power of AI Agentic Workflows**

In today's fast-paced business landscape, organizations are constantly seeking ways to streamline processes and boost productivity. One innovative solution gaining traction is AI agentic workflows. By harnessing the power of artificial intelligence (AI), companies can create dynamic, adaptive workspaces that optimize efficiency and drive success.

**What are AI Agentic Workflows?**

AI agentic workflows refer to a type of workflow management syst